In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
from torch.autograd import Variable

import os
import time
import utils
import numpy as np

from tqdm import tqdm, trange
import matplotlib.pyplot as plt

import networks
import importlib

In [2]:
# seed = 1234
# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# np.random.seed(seed)
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_dtype(torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 10
r = 0.035
sigma = 0.2
T = 1
S_range = [0, int(5*K)]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)
M = 100
N = 5000

# Build Neural Network

In [4]:
net = networks.FeedforwardNeuralNetwork(2, 50, 1, 8) #  Network initialization
net.cuda()

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-7): 7 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [5]:
n_epochs = 5000
lossFunction = nn.MSELoss()
lr = 3e-5
optimizer = optim.Adam(net.parameters(), lr=lr)

x_f_s = torch.tensor(0.).float().to(device).requires_grad_(True)
x_label_s = torch.tensor(0.).float().to(device).requires_grad_(True)
x_data_s = torch.tensor(0.).float().to(device).requires_grad_(True)
optimizer_adam_weight = optim.Adam([x_f_s] + [x_label_s] + [x_data_s], lr=lr*0.0001)

In [6]:
# physical loss samples
samples = {"pde": 5000, "bc":500, "fc":500}

# sample data generated by finite difference method
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor = utils.fdm_data(S_range[-1], T, M, N, "500000sample.csv", device)        

# Modelling


In [7]:
loss_hist = []
start_time = time.time()

for epoch in range(n_epochs):
    
    bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, r, sigma, T, S_range[-1], S_range, t_range, gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    # save training data points to tensor and send to device
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)   
    
    # pde residual loss
    y1_hat = net(n_st_train)
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), 
                retain_graph=True, create_graph=True, only_inputs=True)[0]
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), 
                    create_graph=True, only_inputs=True, allow_unused=True)[0]
    S1 = n_st_train[:, 1].view(-1, 1)
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    # boudary condition loss
    y2_hat = net(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y2_hat)
    
    # sample training data loss
    y3_hat = net(X_train_tensor)
    data_loss = lossFunction(y_train_tensor, y3_hat)
    
    # Backpropagation and Update
    optimizer.zero_grad()
    combined_loss = torch.exp(-x_f_s.detach()) * pde_loss + torch.exp(-x_label_s.detach()) * bc_loss + torch.exp(-x_data_s.detach()) * data_loss + x_data_s + x_label_s + x_f_s
    combined_loss.backward()
    optimizer.step()
    
    # update the weight
    optimizer_adam_weight.zero_grad()
    loss = torch.exp(-x_f_s) * pde_loss.detach() + torch.exp(-x_label_s) * bc_loss.detach() + torch.exp(-x_data_s) * data_loss.detach() + x_data_s + x_label_s + x_f_s
    loss.backward()
    optimizer_adam_weight.step()
    
    # record the loss
    mse_loss = pde_loss + bc_loss + data_loss
    loss_hist.append(mse_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}, data loss: {data_loss.item():.5f}, total loss: {mse_loss.item():.5f}, minimum loss: {min(loss_hist):.5f}')
        print(f'the weight is {torch.exp(-x_f_s.detach()).item():.5f}, {torch.exp(-x_label_s.detach()).item():.5f}. {torch.exp(-x_data_s.detach()).item():.5f}, the parameter is {x_f_s.item():.5f}, {x_label_s.item():.5f}, {x_data_s.item():.5f}')
    pass

end_time = time.time()
print('run time:', end_time - start_time)

0/5000 PDE Loss: 0.00000, BC Loss: 673.62469, data loss: 423.42343, total loss: 1097.04810, minimum loss: 1097.04810
the weight is 1.00000, 1.00000. 1.00000, the parameter is -0.00000, 0.00000, 0.00000
500/5000 PDE Loss: 0.07696, BC Loss: 19.71959, data loss: 14.16921, total loss: 33.96577, minimum loss: 33.96577
the weight is 1.00000, 1.00000. 1.00000, the parameter is -0.00000, 0.00000, 0.00000


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
net.eval()
with torch.no_grad():
    test_outputs = net(X_test_tensor)
    test_loss = lossFunction(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.0018


In [ ]:
for i in range(5000):
    if i % 500 == 0:
        print(loss_hist[i])

1093.721435546875
79.7577896118164
3.6771140098571777
0.07435338199138641
0.027979634702205658
0.011776741594076157
0.005533682648092508
0.003371631260961294
0.0028509057592600584
0.0028797779232263565


In [ ]:
'''
lr = 3e-5
lr = lr
1107.655029296875
91.27940368652344
11.102180480957031
1.2101805210113525
0.004867682233452797
0.003594964276999235
0.0033446408342570066
0.00316116982139647
0.003008157480508089
0.002895315643399954

lr = lr*0.001
1097.4002685546875
47.1202392578125
8.797338485717773
0.13662375509738922
0.003648734651505947
0.003518293611705303
0.0034056329168379307
0.0035889376886188984
0.0031821425072848797
0.0030838181264698505

lr = lr*0.00001
1104.6170654296875
24.64035415649414
12.116193771362305
0.4078262448310852
0.007686748169362545
0.004782408010214567
0.0036287037655711174
0.003150198608636856
0.003064548596739769
0.0028546450193971395
version 2:
1093.721435546875
79.7577896118164
3.6771140098571777
0.07435338199138641
0.027979634702205658
0.011776741594076157
0.005533682648092508
0.003371631260961294
0.0028509057592600584
0.0028797779232263565


lr = lr*0.0000001
1088.1455078125
24.247705459594727
5.023616790771484
0.005683783441781998
0.0031367226038128138
0.0033775828778743744
0.003024038393050432
0.0030635555740445852
0.003273585345596075
0.0029113166965544224
version 2
1088.095947265625
151.35958862304688
9.170495986938477
0.06665617227554321
0.01708376407623291
0.0105251120403409
0.008027860894799232
0.006927900947630405
0.005990810226649046
0.005765970330685377
version 3
1093.8992919921875
29.58930206298828
6.046297073364258
0.027456315234303474
0.010994084179401398
0.006388682406395674
0.004713485017418861
0.003993072081357241
0.004130946937948465
0.003682968905195594
version 4
1096.48853
562.53668
14.12750

lr = lr*0.000000001
the performance is not good and close to normal pinns
1088.857421875
38.67045593261719
6.390939712524414
0.22132942080497742
0.12249445170164108
0.07766617089509964
0.051127832382917404
0.03556811437010765
0.023616353049874306
0.014950141310691833
'''

'\nlr = 3e-5\nlr = lr\n1107.655029296875\n91.27940368652344\n11.102180480957031\n1.2101805210113525\n0.004867682233452797\n0.003594964276999235\n0.0033446408342570066\n0.00316116982139647\n0.003008157480508089\n0.002895315643399954\n\nlr = lr*0.001\n1097.4002685546875\n47.1202392578125\n8.797338485717773\n0.13662375509738922\n0.003648734651505947\n0.003518293611705303\n0.0034056329168379307\n0.0035889376886188984\n0.0031821425072848797\n0.0030838181264698505\n\nlr = lr*0.00001\n1104.6170654296875\n24.64035415649414\n12.116193771362305\n0.4078262448310852\n0.007686748169362545\n0.004782408010214567\n0.0036287037655711174\n0.003150198608636856\n0.003064548596739769\n0.0028546450193971395\n\n\nlr = lr*0.0000001\n1088.1455078125\n24.247705459594727\n5.023616790771484\n0.005683783441781998\n0.0031367226038128138\n0.0033775828778743744\n0.003024038393050432\n0.0030635555740445852\n0.003273585345596075\n0.0029113166965544224\nversion 2\n1088.095947265625\n151.35958862304688\n9.170495986938477